# Test

In [12]:
# ! pip install openai
# ! pip install -q azure_identity
# ! pip install python-dotenv


In [13]:
import os
from dotenv import load_dotenv

from openai import AzureOpenAI
# from azure.identity import DefaultAzureCredential, get_bearer_token_provider
load_dotenv()


True

In [8]:
endpoint = os.getenv("ENDPOINT_URL")
deployment = os.getenv("DEPLOYMENT_NAME")
search_endpoint = os.getenv("SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_KEY")
search_index = os.getenv("SEARCH_INDEX_NAME")

In [5]:
# token_provider = get_bearer_token_provider(
#     DefaultAzureCredential(),
#     "https://cognitiveservices.azure.com/.default")

In [9]:
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=endpoint,
    # azure_ad_token_provider=token_provider,
    api_version="2024-05-01-preview",
)

In [30]:
messages = [
    {
        "role": "system",
        "content": "You are an AI model specialized in financial analysis. Your task is to extract specific financial metrics related to capital expenditure (CapEx) from earnings call transcripts of a company.",
    },
    {
        "role": "user",
        "content": "For Amazon provide a detailed summary for all the quarters and years where sections like CapEx, capital spending, or capital investment is discussed."
    }
    # {
    #     "role": "user",
    #     "content": "Extract the capital expenditure (CapEx) information from the Alpabet's Q4 2022 earnings call transcript. Look for sections where CapEx, capital spending, or capital investment is discussed. Provide the extracted numerical values and any relevant context or statements made by the company's management regarding CapEx. If there are any projections or comparisons to previous periods, include those as well."
    # }
]

In [31]:
completion = client.chat.completions.create(
    model=deployment,
    messages= messages,
    max_tokens=800,
    temperature=0,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": "azureblob-index",
            "semantic_configuration": "call-transcript-semantic-config",
            "query_type": "semantic",
            "fields_mapping": {},
            "in_scope": True,
            "role_information": "You are an AI model specialized in financial analysis. Your task is to extract specific financial metrics related to capital expenditure (CapEx) from earnings call transcripts of a company.",
            "filter": None,
            "strictness": 3,
            "top_n_documents": 5,
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            }
          }
        }]
    }
)

In [32]:
print(completion.choices[0].message.content)

The requested information is not available in the retrieved data. Please try another query or topic.


# Test 2

In [33]:
! pip install azure-search-documents==11.6.0b4 --quiet


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [35]:
from azure.search.documents import SearchClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name="hotels-sample-index",
    credential=credential
)

In [43]:
query ="""
give a summary for amazon q2 2022.
"""

In [46]:
search_results = search_client.search(
    search_text=query,
    top=5,
    select="Description,HotelName,Tags"
 )

In [47]:
sources_formatted = "\n".join([f'{document["HotelName"]}:{document["Description"]}:{document["Tags"]}' for document in search_results])

HttpResponseError: Operation returned an invalid status 'Forbidden'

In [42]:
sources_formatted

''

# test 3

In [1]:
# ! pip install -q openai PyPDF2


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import os
from openai import AzureOpenAI
import PyPDF2

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

file_path = "company/VLO-3Q23-Earnings-Release.pdf"

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.

    Args:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: Extracted text from the PDF.
    """
    text = ""
    with open(pdf_path, "rb") as file:
        pdf_reader = PyPDF2.PdfReader(file)
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
    return text

input_context = extract_text_from_pdf(file_path)

instructions = """
    As a Financial Analyst, you will leverage your expertise to generate tailored Financial Analysis Reports that cater to the specific requirements of clients. This role involves in-depth analysis of financial statements and market data to uncover insights into a company's financial performance and stability.
You will engage directly with clients to gather essential information and refine the report based on their feedback, ensuring that the final product precisely meets their needs and expectations.

Key Objectives:

Analytical Precision: Utilize analytical skills to interpret financial data, identify trends, and detect anomalies.
Effective Communication: Simplify and convey complex financial information in a clear and actionable manner for non-specialist audiences.
Client Focus: Tailor reports dynamically based on client feedback, aligning the analysis with their strategic goals.
Quality Assurance: Maintain the highest standards of quality and integrity in report generation, adhering to established benchmarks for analytical rigor.
Performance Indicators:

The effectiveness of the Financial Analysis Report is measured by its ability to provide actionable insights that support corporate decision-making, identify areas for operational improvement, and evaluate the company's financial health. Success is reflected in the report's contribution to informed investment decisions and strategic planning.
"""

user_content = f"""The report will include the following sections and I want to extract the capital expenditure and other metric like 

Operating Activities "Change in Working Capital",
        "Net Cash from Operating Activities"
Investing Activities has "Acquisition of Fixed Assets & Intangibles",
        "Net Cash from Investing Activities"
Financing Activities includes "Dividends Paid",
        "Cash from (Repayment of) Debt",
        "Net Cash from Financing Activities",
Net Change includes  Net Change in Cash
Metadata includes "Report Date", "Publish Date", "Source"
Profitability Metrics like "EBITDA",
        "Gross Profit Margin",
        "Operating Margin",
        "Net Profit Margin",
        "Return on Equity",
        "Return on Assets",
        "Return On Invested Capital",
Liquidity Metrics include Current Ratio
Solvency Metrics like  "Total Debt", "Liabilities to Equity Ratio", "Debt Ratio",
Cash Flow Metrics like "Free Cash Flow", "Free Cash Flow to Net Income", "Cash Return On Invested Capital",
Other Important Metrics like "Piotroski F-Score", "Net Debt / EBITDA", "Dividend Payout Ratio
Capital Expenditure    

```Document
{input_context}
```
"""

final_content = """Please provide the  company name, year and quarter, capital expenditure value in billions of US dollars in JSON format.
Note: Only Json Data is required no other text is required.
```json

"""

messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": user_content}
]
response = client.chat.completions.create(
    model = os.getenv("DEPLOYMENT_NAME"),
    messages = messages,
)

"""
[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure AI services support this too?"}
    ]
"""
print(response.choices[0].message.content)

messages.append({"role": "assistant", "content":response.choices[0].message.content})
messages.append({"role": "user", "content":final_content})


Based on the provided document, the following financial metrics can be extracted:

- Operating Activities "Change in Working Capital": In the "Investing and Financing Activities" section, there is a line that mentions a "favorable change in working capital" of $33 million in the third quarter of 2023.

- Net Cash from Operating Activities: In the same section, it is mentioned that net cash provided by operating activities was $3.3 billion in the third quarter of 2023.

- Acquisition of Fixed Assets & Intangibles: In the "Investing and Financing Activities" section, it is mentioned that capital investments totaled $394 million in the third quarter of 2023.

- Net Cash from Investing Activities: In the same section, it is mentioned that net cash provided by investing activities was negative $61 million in the third quarter of 2023.

- Dividends Paid: It is mentioned in the document that Valero returned $2.2 billion to stockholders in the third quarter of 2023, of which $360 million was p

In [5]:
response = client.chat.completions.create(
    model = os.getenv("DEPLOYMENT_NAME"),
    messages = messages,
)
print(response.choices[0].message.content)

{
  "company_name": "Valero Energy Corporation",
  "year": 2023,
  "quarter": 3,
  "capital_expenditure": 0.394
}


In [3]:
import PyPDF2

text = ""
with open("company/AMZN-Q1-2024-Earnings-Release.pdf", "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text

text[:50]

'AMAZON.COM ANNOUNCES FIRST QUARTER RESULTS\nSEATTLE'

# test 4

In [1]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [17]:
import os
import io

import PyPDF2
connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
container_name = "earningcalltranscriptscontainer"  # Replace with your container name
container_client = ContainerClient.from_connection_string(
    conn_str=connection_string,
    container_name="earningcalltranscriptscontainer"
)
# file_names = container_client.list_blobs() 

blob_list = container_client.list_blobs()
# file_streams = [container_client.download_blob(blob).readall() for blob in blob_list]
# "company/AMZN-Q1-2024-Earnings-Release.pdf"
text = io.BytesIO()
x = container_client.download_blob("AMZN-Q1-2024-Earnings-Release.pdf").readinto(text)
# text.write(x)

ntext = ""
pdf_reader = PyPDF2.PdfReader(text)
for page in pdf_reader.pages:
    page_text = page.extract_text()
    if page_text:
        ntext += page_text
ntext[:50]
# file_streams = []
# for file_name in file_names:
#     blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
#     downloaded_blob = blob_client.download_blob()
#     file_streams.append(downloaded_blob.content_as_bytes())

'AMAZON.COM ANNOUNCES FIRST QUARTER RESULTS\nSEATTLE'